In [22]:
using MatrixNetworks
using Distributions
using Gadfly
using Clustering
using BenchmarkTools, Compat
using DelimitedFiles
using SparseArrays
using LinearAlgebra
using EllipsoidalGraphEmbedding
using Random

Loadind data sets

In [23]:
# Please choose the data set below

dataset = "PolBlogs"
# dataset = "PowerEU"
# dataset = "Facebook"
#  dataset = "PowerUS"
# dataset = "Internet"
# dataset = "CondMat2003"
#dataset = "Pokec"

# Initialization
A = 0

# nb iterations projected power method
n_it_PPM = 30000

# relative objective variation
t = 1e-8

# nb of times vector partition is repeated
n_rep_vec_part = 10 

# upper bound on nb updates for vector partition
n_updates = 50 

# shape of embedding
shape = "Ellipsoidal"

# number of columns of initial guess
d0 = 10 

# number of centroids thrown in embedding
n_clusters = 100

# nb of times vector partition is repeated
n_it_vec_part = 100 

# number of centroids thrown in embedding
n_clusters = 100

# Loading data
if dataset == "PolBlogs"

    A0 = readdlm("data/PolBlogs/PolBlogsAdj.txt")
    A = largest_cc_from_adj(A0); A0 = 0;
    dim_embed_spectral = 2

elseif dataset == "PowerEU"

    list = readdlm("data/PowerEU/ElistPowerEU.txt")
    A = adj_matrix_cc(list)
    dim_embed_spectral = 50

elseif dataset == "Facebook"

    list = readdlm("data/Facebook/facebook-gender_edges.txt")
    list = list .+ 1 # indices start from 0
    A = adj_matrix_cc(list)
    dim_embed_spectral = 8

elseif dataset == "PowerUS"

    list = readdlm("data/PowerUS/ElistPowerUS.txt")
    A = adj_matrix_cc(list)
    dim_embed_spectral = 80

elseif dataset == "Wikivote"

    list = readdlm("data/Wikivote/wiki-Vote.txt")
    A = adj_matrix_cc(list)
    dim_embed_spectral = 3

elseif dataset == "Internet"

    list = readdlm("data/Internet/InternetEList.txt")
    list = list .+ 1 # indices start from 0
    A = adj_matrix_cc(list)
    dim_embed_spectral = 18

elseif dataset == "CondMat2003"

    list = readdlm("data/CondMat2003/CondMat2003EList.txt")
    list = list .+ 1 # indices start from 0
    A = adj_matrix_cc(list)
    dim_embed_spectral = 80

elseif dataset == "Pokec"

    list = readdlm("data/Pokec/soc-pokec-relationships.txt")
    list = list .+ 1 # indices start from 0
    A = adj_matrix_cc(list)
    dim_embed_spectral = 50

end
println(dataset)

N = size(A,1)
community = zeros(Int64,N,1)
x_embed,community,singular =  @time sphere_embed_cluster(A, n_it_PPM, t, n_clusters, n_it_vec_part, n_updates, shape, d0)
# run twice to get timings without compilation time
println(dataset)
println("embedding effective dim= ",dim_eff(singular,0.01))


PolBlogs
 ------- Acc Projected Power Iteration -------


The iteration has become stationary after 81 iterations
  0.145654 seconds (2.21 k allocations: 69.073 MiB, 28.38% gc time)
 -------------- Clustering ------- 
dimension of embedding used for clustering: 10
Number of updates: 

3
Modularity: 0.4314366107288681
Number of communities: 7
 -------------------------------------------- 
The first 5 squared singular values divided by N : 
[0.9862353557266231, 0.012559610676315553, 0.0008003391642211193, 0.00027157310507087886, 0.00012670029144277791]
 -------------------------------------------- 
  1.568013 seconds (1.28 M allocations: 822.910 MiB, 24.60% gc time)
PolBlogs
embedding effective dim= 2


Spectral embedding

In [20]:
# an approximate Power Method with Krylov subspace isi implemented in spectral_embed_cluster
tol = 1e-06
it_max = 1000
# spectral embedding
community = zeros(Int64,N,1)
x_embed, community, eigenvalues = @time spectral_embed_cluster(A, it_max, tol, n_clusters, n_rep_vec_part, n_updates, dim_embed_spectral)
println(dataset)

InterruptException: InterruptException: